<a href="https://colab.research.google.com/github/yumixxxx/Tbrain/blob/main/Data%20Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import shutil
import torch
import zipfile
from zipfile import ZipFile
from PIL import Image
from PIL import ImageFile
from torchvision import transforms

1. Unzip Training Dataset_v5.zip


In [ ]:
test_file_name = "Training Dataset_v5.zip"
with ZipFile(test_file_name, 'r') as zip:
    zip.printdir()
    zip.extractall('Total_Dataset')

2. Separate images and labels

In [ ]:
#separate images and labels
src = 'Total_Dataset/train'
folder_image = 'All_images'
folder_label = 'All_labels'

#img
file_img = glob.glob(os.path.join(src, "*.png"), recursive=True)
for f1 in file_img:
    dst_image = os.path.join(folder_image, os.path.basename(f1))
#     print(dst_image)
    shutil.copy(f1, dst_image)

#label
file_lab = glob.glob(os.path.join(src, "*.txt"), recursive=True)
for f2 in file_lab:
    dst_label = os.path.join(folder_label, os.path.basename(f2))
#     print(dst_label)
    shutil.copy(f2, dst_label)

3. Resize and Unsharp images
4. Convert the labels into demanded yolo form

In [ ]:
import albumentations as A
import cv2
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
transform = A.Compose([
    A.Resize(width=1280, height=1280),
    A.UnsharpMask(p=1)
])

In [ ]:
def normalize(bboxes:np.array, img_w, img_h):
  bboxes[:,1] = (bboxes[:,1] + bboxes[:,3] / 2) / img_w
  bboxes[:,2] = (bboxes[:,2] + bboxes[:,4] / 2) / img_h
  bboxes[:,3] = bboxes[:,3] / img_w
  bboxes[:,4] = bboxes[:,4] / img_h
  return bboxes

file_images = glob.glob(os.path.join("All_images","*.png"), recursive=True)
file_images.sort()
file_labels = glob.glob(os.path.join("All_labels","*.txt"), recursive=True)
file_labels.sort()

for image_path, label_path in zip(file_images, file_labels):   
  #image
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  label_df = pd.read_csv(label_path, header=None, names=['c', 'x', 'y', 'w', 'h'])
  label_df['class'] = label_df['c']
  bboxes = label_df.drop(['c'], axis=1).to_numpy().tolist()

  for i in range(1,3):
    transformed = transform(image=image, bboxes = bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    
    if len(transformed_bboxes) != 0:
      #transform and save images
      im = Image.fromarray(transformed_image)
      im.save(image_path)
      
      #transform and save labels
      transformed_bboxes = np.array(transformed_bboxes)
      classes = transformed_bboxes[:,[-1]]
      transformed_bboxes = np.concatenate([classes, transformed_bboxes], axis=1)[:,:-1]

      transformed_bboxes = normalize(transformed_bboxes, 1280, 1280)
      transformed_bboxes = pd.DataFrame(transformed_bboxes)
      transformed_bboxes[0] = transformed_bboxes[0].astype('int64')
      transformed_bboxes.to_csv(label_path, sep=' ', header=False, index=False) #save
    else:
      with open(label_path, 'w') as f: #save
        f.write('')

5. Separate training data and validation data

In [ ]:
#image
file1 = glob.glob(os.path.join("All_images", "*.png"), recursive=True)
file1.sort()

for file_path in file1:
    img = Image.open(file_path)
    if(int(file_path[-8:-4])<801):
        shutil.copy(file_path, "custom_data/images/train")
    else:
        shutil.copy(file_path, "custom_data/images/val")
        
        
#label
file2 = glob.glob(os.path.join("normalize_labels", "*.txt"), recursive=True)
file2.sort()

for file_path in file2:
    if(int(file_path[-8:-4])<801):
        shutil.copy(file_path, "custom_data/labels/train")
    else:
        shutil.copy(file_path, "custom_data/labels/val")